In [1]:
import findspark
findspark.init()


In [2]:
findspark.find()


'C:\\spark\\spark-2.4.4-bin-hadoop2.7'

In [3]:
import pyspark

In [4]:
from pyspark.sql.window import Window
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder\
    .appName("NNS")\
    .getOrCreate()

In [6]:
spark = SparkSession.builder.master("local[1]").appName("Demo").getOrCreate()

In [7]:
df = spark.read.csv("C:/practice/pyspark/nns_pyspark_data_pipeline_example/srcdata/capitalcitypopulation.csv")
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)



In [8]:
df.show()

+-------------+-----------+----------+-------+
|          _c0|        _c1|       _c2|    _c3|
+-------------+-----------+----------+-------+
|         city|countrycode|population|   unit|
|        Tokyo|        JPN|       210|Million|
|    New Delhi|        IND|       175|Million|
|Washington DC|        USA|       115|Million|
|        Paris|        FRA|        75|Million|
|       Moscow|        RUS|       125|Million|
|       London|        GBR|       120|Million|
|         Rome|        ITA|        80|Million|
+-------------+-----------+----------+-------+



In [9]:
df.createOrReplaceTempView("PERSON_DATA")

In [10]:
df2 = spark.sql("SELECT * from PERSON_DATA")

In [11]:
df2.show()

+-------------+-----------+----------+-------+
|          _c0|        _c1|       _c2|    _c3|
+-------------+-----------+----------+-------+
|         city|countrycode|population|   unit|
|        Tokyo|        JPN|       210|Million|
|    New Delhi|        IND|       175|Million|
|Washington DC|        USA|       115|Million|
|        Paris|        FRA|        75|Million|
|       Moscow|        RUS|       125|Million|
|       London|        GBR|       120|Million|
|         Rome|        ITA|        80|Million|
+-------------+-----------+----------+-------+



In [12]:
groupDF = spark.sql("SELECT _c0, count(*) from PERSON_DATA group by _c0")

In [13]:
groupDF.show()

+-------------+--------+
|          _c0|count(1)|
+-------------+--------+
|       Moscow|       1|
|       London|       1|
|        Tokyo|       1|
|        Paris|       1|
|         city|       1|
|Washington DC|       1|
|         Rome|       1|
|    New Delhi|       1|
+-------------+--------+



In [14]:
from pyspark.sql.functions import desc,rank

In [15]:
winSpec = Window.partitionBy("_c0").orderBy(desc("_c2"))

In [16]:
f_df = df2.withColumn("rank", rank().over(winSpec))

In [17]:
f_df.show()

+-------------+-----------+----------+-------+----+
|          _c0|        _c1|       _c2|    _c3|rank|
+-------------+-----------+----------+-------+----+
|       Moscow|        RUS|       125|Million|   1|
|       London|        GBR|       120|Million|   1|
|        Tokyo|        JPN|       210|Million|   1|
|        Paris|        FRA|        75|Million|   1|
|         city|countrycode|population|   unit|   1|
|Washington DC|        USA|       115|Million|   1|
|         Rome|        ITA|        80|Million|   1|
|    New Delhi|        IND|       175|Million|   1|
+-------------+-----------+----------+-------+----+



In [18]:
v_city="Paris"

In [21]:
f_df.write.option("header",True) \
        .partitionBy("_c1") \
        .mode("overwrite") \
        .csv("c:/11/country_pop")

In [23]:
circuit_df = spark.read.option("header","true").csv("./data/raw/circuits.csv")

In [24]:
circuit_df.show()

+---------+--------------+--------------------+------------+---------+--------+---------+---+--------------------+
|circuitId|    circuitRef|                name|    location|  country|     lat|      lng|alt|                 url|
+---------+--------------+--------------------+------------+---------+--------+---------+---+--------------------+
|        1|   albert_park|Albert Park Grand...|   Melbourne|Australia|-37.8497|  144.968| 10|http://en.wikiped...|
|        2|        sepang|Sepang Internatio...|Kuala Lumpur| Malaysia| 2.76083|  101.738| 18|http://en.wikiped...|
|        3|       bahrain|Bahrain Internati...|      Sakhir|  Bahrain| 26.0325|  50.5106|  7|http://en.wikiped...|
|        4|     catalunya|Circuit de Barcel...|    Montmeló|    Spain|   41.57|  2.26111|109|http://en.wikiped...|
|        5|      istanbul|       Istanbul Park|    Istanbul|   Turkey| 40.9517|   29.405|130|http://en.wikiped...|
|        6|        monaco|   Circuit de Monaco| Monte-Carlo|   Monaco| 43.7347| 

In [26]:
circuit_df.select("circuitId","circuitRef","name").show()

+---------+--------------+--------------------+
|circuitId|    circuitRef|                name|
+---------+--------------+--------------------+
|        1|   albert_park|Albert Park Grand...|
|        2|        sepang|Sepang Internatio...|
|        3|       bahrain|Bahrain Internati...|
|        4|     catalunya|Circuit de Barcel...|
|        5|      istanbul|       Istanbul Park|
|        6|        monaco|   Circuit de Monaco|
|        7|    villeneuve|Circuit Gilles Vi...|
|        8|   magny_cours|Circuit de Nevers...|
|        9|   silverstone| Silverstone Circuit|
|       10|hockenheimring|      Hockenheimring|
|       11|   hungaroring|         Hungaroring|
|       12|      valencia|Valencia Street C...|
|       13|           spa|Circuit de Spa-Fr...|
|       14|         monza|Autodromo Naziona...|
|       15|    marina_bay|Marina Bay Street...|
|       16|          fuji|       Fuji Speedway|
|       17|      shanghai|Shanghai Internat...|
|       18|    interlagos|Autódromo José

In [27]:
from pyspark.sql.functions import col

In [28]:
circuit_df.select(col("circuitId").alias("Cicuit-ID"),"circuitRef","name").show()

+---------+--------------+--------------------+
|Cicuit-ID|    circuitRef|                name|
+---------+--------------+--------------------+
|        1|   albert_park|Albert Park Grand...|
|        2|        sepang|Sepang Internatio...|
|        3|       bahrain|Bahrain Internati...|
|        4|     catalunya|Circuit de Barcel...|
|        5|      istanbul|       Istanbul Park|
|        6|        monaco|   Circuit de Monaco|
|        7|    villeneuve|Circuit Gilles Vi...|
|        8|   magny_cours|Circuit de Nevers...|
|        9|   silverstone| Silverstone Circuit|
|       10|hockenheimring|      Hockenheimring|
|       11|   hungaroring|         Hungaroring|
|       12|      valencia|Valencia Street C...|
|       13|           spa|Circuit de Spa-Fr...|
|       14|         monza|Autodromo Naziona...|
|       15|    marina_bay|Marina Bay Street...|
|       16|          fuji|       Fuji Speedway|
|       17|      shanghai|Shanghai Internat...|
|       18|    interlagos|Autódromo José

In [29]:
circuit_df.withColumnRenamed("circuitId","Cicuit-ID").show()

+---------+--------------+--------------------+------------+---------+--------+---------+---+--------------------+
|Cicuit-ID|    circuitRef|                name|    location|  country|     lat|      lng|alt|                 url|
+---------+--------------+--------------------+------------+---------+--------+---------+---+--------------------+
|        1|   albert_park|Albert Park Grand...|   Melbourne|Australia|-37.8497|  144.968| 10|http://en.wikiped...|
|        2|        sepang|Sepang Internatio...|Kuala Lumpur| Malaysia| 2.76083|  101.738| 18|http://en.wikiped...|
|        3|       bahrain|Bahrain Internati...|      Sakhir|  Bahrain| 26.0325|  50.5106|  7|http://en.wikiped...|
|        4|     catalunya|Circuit de Barcel...|    Montmeló|    Spain|   41.57|  2.26111|109|http://en.wikiped...|
|        5|      istanbul|       Istanbul Park|    Istanbul|   Turkey| 40.9517|   29.405|130|http://en.wikiped...|
|        6|        monaco|   Circuit de Monaco| Monte-Carlo|   Monaco| 43.7347| 

In [37]:
from pyspark.sql.functions import current_timestamp

circuit_df.withColumn("Date", current_timestamp()).show()

+---------+--------------+--------------------+------------+---------+--------+---------+---+--------------------+--------------------+
|circuitId|    circuitRef|                name|    location|  country|     lat|      lng|alt|                 url|                Date|
+---------+--------------+--------------------+------------+---------+--------+---------+---+--------------------+--------------------+
|        1|   albert_park|Albert Park Grand...|   Melbourne|Australia|-37.8497|  144.968| 10|http://en.wikiped...|2023-02-28 17:46:...|
|        2|        sepang|Sepang Internatio...|Kuala Lumpur| Malaysia| 2.76083|  101.738| 18|http://en.wikiped...|2023-02-28 17:46:...|
|        3|       bahrain|Bahrain Internati...|      Sakhir|  Bahrain| 26.0325|  50.5106|  7|http://en.wikiped...|2023-02-28 17:46:...|
|        4|     catalunya|Circuit de Barcel...|    Montmeló|    Spain|   41.57|  2.26111|109|http://en.wikiped...|2023-02-28 17:46:...|
|        5|      istanbul|       Istanbul Park| 

In [41]:
from pyspark.sql.functions import lit

circuit_df.withColumn("const_col", lit("XYZ")).show()

+---------+--------------+--------------------+------------+---------+--------+---------+---+--------------------+---------+
|circuitId|    circuitRef|                name|    location|  country|     lat|      lng|alt|                 url|const_col|
+---------+--------------+--------------------+------------+---------+--------+---------+---+--------------------+---------+
|        1|   albert_park|Albert Park Grand...|   Melbourne|Australia|-37.8497|  144.968| 10|http://en.wikiped...|      XYZ|
|        2|        sepang|Sepang Internatio...|Kuala Lumpur| Malaysia| 2.76083|  101.738| 18|http://en.wikiped...|      XYZ|
|        3|       bahrain|Bahrain Internati...|      Sakhir|  Bahrain| 26.0325|  50.5106|  7|http://en.wikiped...|      XYZ|
|        4|     catalunya|Circuit de Barcel...|    Montmeló|    Spain|   41.57|  2.26111|109|http://en.wikiped...|      XYZ|
|        5|      istanbul|       Istanbul Park|    Istanbul|   Turkey| 40.9517|   29.405|130|http://en.wikiped...|      XYZ|


In [43]:
circuit_df.write.parquet("C:/11/c")